In [2]:
from sqlalchemy import create_engine, Column, Integer, String, Date, Time, ForeignKey, DECIMAL
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker
from werkzeug.security import generate_password_hash, check_password_hash
import json

Base = declarative_base()

class Funcionario(Base):
    __tablename__ = 'funcionario'
    id_funcionario = Column(Integer, primary_key=True, autoincrement=True)
    nome = Column(String(255), nullable=False)
    contato = Column(String(255), nullable=False)
    cargo = Column(String(255), nullable=False)

class Servico(Base):
    __tablename__ = 'servico'
    id_servico = Column(Integer, primary_key=True, autoincrement=True)
    descricao = Column(String(255), nullable=False)
    preco = Column(DECIMAL(10, 2), nullable=False)
    duracao = Column(Time, nullable=False)

class Cliente(Base):
    __tablename__ = 'cliente'
    id_cliente = Column(Integer, primary_key=True, autoincrement=True)
    nome = Column(String(255), nullable=False)
    email = Column(String(255), nullable=False)
    telefone = Column(String(255), nullable=False)
    endereco = Column(String(255), nullable=False)
    dt_nascimento = Column(Date, nullable=False)
    genero = Column(String(15), nullable=False)

class Agendamento(Base):
    __tablename__ = 'agendamento'
    id_agendamento = Column(Integer, primary_key=True, autoincrement=True)
    id_cliente = Column(Integer, ForeignKey('cliente.id_cliente'))
    id_servico = Column(Integer, ForeignKey('servico.id_servico'))
    id_funcionario = Column(Integer, ForeignKey('funcionario.id_funcionario'))
    data = Column(Date)
    hora = Column(Time)
    cliente = relationship("Cliente")
    servico = relationship("Servico")
    funcionario = relationship("Funcionario")

class Usuario(Base):
    __tablename__ = 'usuario'
    id_usuario = Column(Integer, primary_key=True, autoincrement=True)
    nome = Column(String(255), nullable=False)
    email = Column(String(255), nullable=False, unique=True)
    senha = Column(String(255), nullable=False)

    def set_senha(self, senha):
        self.senha = generate_password_hash(senha)

    def check_senha(self, senha):
        return check_password_hash(self.senha, senha)

    def to_dict(self):
        return {
            'id_usuario': self.id_usuario,
            'nome': self.nome,
            'email': self.email
        }

with open('credenciais.json', 'r') as f:
    config = json.load(f)

# Configurar a conexão com o banco de dados
engine = create_engine(config['database']['connection_string'])
Base.metadata.create_all(engine)

# Criar uma sessão
Session = sessionmaker(bind=engine)
session = Session()


C:\Users\vinit\AppData\Local\Temp\ipykernel_8084\2938928021.py:7: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [3]:
def add_funcionario(nome, contato, cargo):
    novo_funcionario = Funcionario(nome=nome, contato=contato, cargo=cargo)
    session.add(novo_funcionario)
    session.commit()

def add_servico(descricao, preco, duracao):
    novo_servico = Servico(descricao=descricao, preco=preco, duracao=duracao)
    session.add(novo_servico)
    session.commit()

def add_cliente(nome, email, telefone, endereco, dt_nascimento, genero):
    novo_cliente = Cliente(nome=nome, email=email, telefone=telefone, endereco=endereco, dt_nascimento=dt_nascimento, genero=genero)
    session.add(novo_cliente)
    session.commit()

def add_agendamento(id_cliente, id_servico, id_funcionario, data, hora):
    novo_agendamento = Agendamento(id_cliente=id_cliente, id_servico=id_servico, id_funcionario=id_funcionario, data=data, hora=hora)
    session.add(novo_agendamento)
    session.commit()

def add_usuario(nome, email, senha):
    if session.query(Usuario).filter_by(email=email).first():
        return "Email já registrado"
    novo_usuario = Usuario(nome=nome, email=email)
    novo_usuario.set_senha(senha)
    session.add(novo_usuario)
    session.commit()
    return "Usuário cadastrado com sucesso"


In [ ]:
def get_funcionarios():
    return session.query(Funcionario).all()

def get_servicos():
    return session.query(Servico).all()

def get_clientes():
    return session.query(Cliente).all()

def get_agendamentos():
    return session.query(Agendamento).all()

def get_usuarios():
    return session.query(Usuario).all()


In [ ]:
def update_funcionario(id_funcionario, nome=None, contato=None, cargo=None):
    funcionario = session.query(Funcionario).filter_by(id_funcionario=id_funcionario).first()
    if funcionario:
        if nome:
            funcionario.nome = nome
        if contato:
            funcionario.contato = contato
        if cargo:
            funcionario.cargo = cargo
        session.commit()

def update_servico(id_servico, descricao=None, preco=None, duracao=None):
    servico = session.query(Servico).filter_by(id_servico=id_servico).first()
    if servico:
        if descricao:
            servico.descricao = descricao
        if preco:
            servico.preco = preco
        if duracao:
            servico.duracao = duracao
        session.commit()

def update_cliente(id_cliente, nome=None, email=None, telefone=None, endereco=None, dt_nascimento=None, genero=None):
    cliente = session.query(Cliente).filter_by(id_cliente=id_cliente).first()
    if cliente:
        if nome:
            cliente.nome = nome
        if email:
            cliente.email = email
        if telefone:
            cliente.telefone = telefone
        if endereco:
            cliente.endereco = endereco
        if dt_nascimento:
            cliente.dt_nascimento = dt_nascimento
        if genero:
            cliente.genero = genero
        session.commit()



In [ ]:
def delete_funcionario(id_funcionario):
    funcionario = session.query(Funcionario).filter_by(id_funcionario=id_funcionario).first()
    if funcionario:
        session.delete(funcionario)
        session.commit()

def delete_servico(id_servico):
    servico = session.query(Servico).filter_by(id_servico=id_servico).first()
    if servico:
        session.delete(servico)
        session.commit()

def delete_cliente(id_cliente):
    cliente = session.query(Cliente).filter_by(id_cliente=id_cliente).first()
    if cliente:
        session.delete(cliente)
        session.commit()

def delete_agendamento(id_agendamento):
    agendamento = session.query(Agendamento).filter_by(id_agendamento=id_agendamento).first()
    if agendamento:
        session.delete(agendamento)
        session.commit()

